# ING Insights


Notebook used to categorize and automatically analyse transations data from ING Bank Account. This account is aggregation transaction information from all accounts in ING Bank as well as Millennium.

Analysis should be used to aggregate per month infomrations infomrations about spending, mainly from accounts `Wspólne`, `Osobiste`, `Millennium`.


**TODO**
- [ ] Export to Excel
- [ ] Different data analysis options
- [ ] Make mark in Excel file which rows were analysed by the tool. Alternatively records could be removed from `df`

## 01 - Importing data

Import data from ING Bank history export. Class used to import the data `TransactionDataLoader` consisting fetching mechanism with data sanitization finctionalities.

In [ ]:
from pathlib import Path

from data_loader import *

csv_import_1 = Path("data/Lista_transakcji_nr_0202366310_201224.csv")
csv_import_2 = Path("data/20241220_lista_transkcji_ing_mill_90.csv")
csv_import_3 = Path("data/ing_12_2024.csv")

df_import = TransactionDataLoader(csv_import_3).get_data()
print(f"Successfully imported dataframe with size {df_import.shape}\n\n")



## 02 - Data filtering


Filter only usefully columns for further data analysis. This will be the base for further analysis of data.

1. Columns should contain set of information which will be used for further analysis.
2. Values of `Kwota transakcji (waluta rachunku)` should be __negative__. Only expenses to be analysed.
3. Exclude own transfers.


In [ ]:
selected_columns = ['Data transakcji', 'Dane kontrahenta', 'Tytuł', 'Kwota transakcji (waluta rachunku)', 'Bank']

# 1. Only useful columns used for analysis
df_c = df_import[selected_columns]

# 2. Remove rows with negative values in column ''Kwota transakcji (waluta rachunku)''
df_neg = df_c[df_c['Kwota transakcji (waluta rachunku)'] < 0]

# 3. Exclude own transfers
df_not = df_neg[df_neg["Tytuł"].str.contains("Przelew własny|Own transfer") == False]

# TODO: Lowercase all columns
# 4. Filter out not relevant transactions.
df = df_not[df_not["Dane kontrahenta"].str.contains("Albert|ALBERT|Raty|RATA|RATY|Rata") == False]

## 03 - Categorization


Define categories for analysis and fill the data based on filter definition applied to `Dane kontrachenta` in `df` data frame.

`ExpenseCategory` dataclass contains atrribute for sotring data filtered by applied filter.

Categorized expance data will be storred in `expense_category_list`


In [ ]:
"""
Input data for analysis: df

ExpenceCategiries dataclass instance containing objects of expense category.
 - name: str
 - filter: str
 - sum: Optional[float]
 - data_frame: Optional[pd.DataFrame]
"""
expense_category_list = [
    ExpenseCategory(name="Zakupy spożywcze",
                    filter="LIDL|BIED|ZAB|PIEKARNIA|Zygula|Piekarstwo|CARREFOUR|FAMILIJNA|CENTRUM WINA|NETTO|AUCHAN"),
    ExpenseCategory(name="Kosmetyki",
                    filter="ROSSMANN|notino"),
    ExpenseCategory(name="Restauracje",
                    filter="PIAZZA|COCKPEAT|CHLEBOTEKA|WHISKEYINTHEJAR|TUTTI|NEWPORT CAFE|SHRIMP HOUSE"),
    ExpenseCategory(name="Zdrowie",
                    filter="APTEKA|LUX MED|SUPER-PHARM"),
    ExpenseCategory(name="Transport",
                    filter="SHELL|Myjnia|ORLEN|WROCLAVIA PARKING|Taxi|APCOA|Autopay|STACJA PALIW|CIRCLE K")
]

# Filter df dataframe and fill each ExpenseCategory data class attributes.
for i in range(len(expense_category_list)):
    expense_category_list[i].data_frame = df[
        df["Dane kontrahenta"].str.contains(expense_category_list[i].filter) == True]

    expense_category_list[i].sum = abs(
        round(float(expense_category_list[i].data_frame["Kwota transakcji (waluta rachunku)"].sum()), 2))

### 3.1 - Categorization helper

Based on data which is not matching any filter build better filter for transactions in account statement.

In [ ]:
sep = "|"
flt_t = []

for exp in expense_category_list:
    flt_t.append(exp.filter)

# Transactions not matching defined filters.
n_df = df[df["Dane kontrahenta"].str.contains(sep.join(flt_t)) == False]

print(
    f"\x1b[31m"  # Colour output red
    f"Transactions not categorized: {n_df.shape[0]} with total sum of: "
    f"{round(n_df["Kwota transakcji (waluta rachunku)"].sum(), 2)} PLN"
    f"\x1b[0m")
n_df


## 04 - Plotting

Transform `expense_category_list` to data frame for ploting purpose.

In [ ]:
from dataclasses import asdict

expense_dicts = [asdict(expense) for expense in expense_category_list]
plt_df = pd.DataFrame(expense_dicts)


Build plot to demonstrate expanse distribution.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

categories = plt_df['name']
values = plt_df['sum']

# Sort data by values
sorted_indices = np.argsort(values)[::-1]  # Descending order
categories_sorted = [categories[i] for i in sorted_indices]
values_sorted = [values[i] for i in sorted_indices]

# Add data labels
for i, value in enumerate(values_sorted):
    plt.text(i, value + 0.5,  # Position slightly above the bar
             str(value),  # Label text
             ha='center',  # Horizontal alignment
             va='bottom')  # Vertical alignment

# Plot the bar chart
bars = plt.bar(categories_sorted, values_sorted)

# Configure plot
plt.xticks(rotation=-60)
plt.title('Analiza transakcji w Banku ING')

# Show the plot
plt.show()

# plt_df sorted by biggest sum value
plt_df.sort_values(by='sum', ascending=False)[['name', 'sum']]

## 99 - Debug

Debug categorized data based on filters defined. Use index <> expace mapping from Plotting section.

In [ ]:
# Verification of category mapping, id of expense category above.

expense_category_list[0].data_frame
